# Determination of ratio D for different density function's slopes

We are going to perform the same calculations of the previous notebook but now changing the slope of the linear density function. We are especting that D must increase for higher slopes because the system gets farther than the constant density picture as the slope gets higher.

**Import packages**

In [1]:
from __future__ import division
import time
import numpy as np
from tesseroid_density import tesseroid
from fatiando.mesher import Tesseroid, TesseroidMesh
from fatiando import gridder
import matplotlib.pyplot as plt

%matplotlib inline

**Creation of the computation grid**

In [2]:
area = (-30, 30, -30, 30)
shape = (20, 20)
h = 10e3
lons, lats, heights = gridder.regular(area, shape, z=h)

**Bounds and D ratio of the Tesseroid Mesh**

In [3]:
D_high = 8
w, e, s, n, top, bottom = -10, 10, -10, 10, -3000, -50000

**Definition of functions**


In [4]:
def optimal_nr():
    "Single computation point"
    lons_single = np.array([0.])
    lats_single = np.array([0.])
    heights_single = np.array([h])
    
    "Initialization of the loop"
    potential_single_point = []
    nr_values = []
    nr = 0
    res = 1e42
    while True:
        nr += 1
        mesh_shape = (nr, 1, 1)
        model_mesh = TesseroidMesh((w, e, s, n, top, bottom), mesh_shape)
        
        density_list = []
        for tess in model_mesh:
            lon1, lon2, lat1, lat2, r1, r2 = tess.get_bounds()
            density_list.append(density_linear(0.5*(lon1 + lon2),
                                               0.5*(lat1 + lat2), 0.5*(r1 + r2)))
        model_mesh.addprop("density", density_list)
        res_old = res
        res = getattr(tesseroid, 'potential')(lons_single,
                                              lats_single,
                                              heights_single,
                                              model_mesh,
                                              ratio=D_high)[0]
        potential_single_point.append(res)
        nr_values.append(nr)
        diff = 100*abs((res-res_old)/res_old)
        if diff < 1e-9:
            break
    return nr


def true_potential_values():
    "We set the density of each tesseroid in the mesh as the"
    "value of the linear density function in its center"
    model_const = TesseroidMesh((w, e, s, n, top, bottom), mesh_shape)
    density_list = []
    for tess in model_const:
        lon1, lon2, lat1, lat2, r1, r2 = tess.get_bounds()
        density_list.append(density_linear(0.5*(lon1 + lon2),
                                           0.5*(lat1 + lat2),
                                           0.5*(r1 + r2)))
    model_const.addprop("density", density_list)
    true_potential = getattr(tesseroid, 'potential')(lons, lats,
                                                     heights, model_const,
                                                     ratio=D_high)
    return true_potential


def best_D_value():
    # Calculate error for each distance-size ratio D
    errors = []
    model = Tesseroid(w, e, s, n, top, bottom, props={"density":density_linear})
    D = 0
    while True:
        D += 0.5
        st = time.time()
        result = getattr(tesseroid, 'potential')(lons, lats, heights, [model], ratio=D)
        print D, "time: ", time.time() - st, "s"
        diff = 100*np.max(np.abs((result - true_potential)/true_potential))
        if diff < 1e-1:
            break
    return D

**Slopes loop**

In [5]:
slopes = [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2]
D_values = []

for a in slopes:
    print "Slope: ", a
    
    "Definition of linear density with current slope"
    def density_linear(lon, lat, height):
        return a*height

    "Get optimal nr"
    nr_optimal = optimal_nr()
    mesh_shape = (nr_optimal, 1, 1)

    print "nr: ", nr_optimal
    
    "Get the true values for the potential"
    true_potential = true_potential_values()
    
    "Get the best D for this slope"
    D_new = best_D_value()
    D_values.append(D_new)
    print "D: ", D_new

Slope:  0.0001
nr:  941
0.5 time:  0.0248429775238 s
1.0 time:  0.0112519264221 s
1.5 time:  0.022185087204 s
D:  1.5
Slope:  0.001
nr:  941
0.5 time:  0.00247001647949 s
1.0 time:  0.00742316246033 s
1.5 time:  0.0201780796051 s
D:  1.5
Slope:  0.01
nr:  941
0.5 time:  0.00291585922241 s
1.0 time:  0.00704288482666 s
1.5 time:  0.021281003952 s
D:  1.5
Slope:  0.1
nr:  941
0.5 time:  0.00271797180176 s
1.0 time:  0.00703310966492 s
1.5 time:  0.0205919742584 s
D:  1.5
Slope:  1
nr:  941
0.5 time:  0.00257802009583 s
1.0 time:  0.00760388374329 s
1.5 time:  0.0215308666229 s
D:  1.5
Slope:  10.0
nr:  941
0.5 time:  0.00239205360413 s
1.0 time:  0.00706815719604 s
1.5 time:  0.0200729370117 s
D:  1.5
Slope:  100.0
nr:  941
0.5 time:  0.0266401767731 s
1.0 time:  0.00992584228516 s
1.5 time:  0.0230929851532 s
D:  1.5
